## Установка библиотек

In [ ]:
# Обновляем datasets для избежания ошибки при использовании load_dataset
!pip install --upgrade datasets

# Устанавливаем фреймворк fireducks - быстрая библиотека DataFrame, разработанная для замены pandas, особенно в тех случаях, когда требуется повышенная скорость обработки данных
!pip install fireducks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

## Импорт бибилотек и модулей

In [ ]:
# Глубокое обучение
import torch

# Для генерации случайных чисел
import random

# Линейная алгебра
import numpy as np

# Pandas
import fireducks.pandas as pd

# Загрузка датасета
from datasets import load_dataset, DatasetDict

# Трансформеры
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM

# Отключим мешаюшие предупреждения
import warnings
warnings.filterwarnings("ignore")

## Загрузка модели

Используем русскоязычную модель GPT от Сбера размера medium `sberbank-ai/rugpt3medium_based_on_gpt2`, чтобы она уместилась на GPU. Также укажем библиотеке pytorch, что вычисления мы будем проводить на графическом процессоре с поддержкой `cuda`:

In [ ]:
DEVICE = torch.device("cuda:0")

# Загрузка и инициализации модели и токенизатора
model_name = "ai-forever/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)

Чтобы проверить, что используемый объект tokenizer действительно поддерживается используется его атрибут is_fast:

In [ ]:
tokenizer.is_fast

True

## Загрузка и подготовка датасета

В качестве датасета будем использовать **MLSUM**, крупномасштабный набор данных для многоязычной суммаризации. Данные извлечены из онлайн-газет и содержат более 1,5 млн пар статья/резюме на пяти различных языках - французском, немецком, испанском, русском и турецком. Загрузим **RU** данные:

In [ ]:
dataset = load_dataset("mlsum", "ru", trust_remote_code=True)

Структура данных:

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 25556
    })
    validation: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 750
    })
    test: Dataset({
        features: ['text', 'summary', 'topic', 'url', 'title', 'date'],
        num_rows: 757
    })
})

Посмотрим на пример данных:

In [ ]:
print("Text: ", dataset["train"][0]["text"])
print("Summary: ", dataset["train"][0]["summary"])
print("Topic: ", dataset["train"][0]["topic"])
print("URL: ", dataset["train"][0]["url"])
print("Title: ", dataset["train"][0]["title"])
print("Date: ", dataset["train"][0]["date"])

Text:  Сладострастник в течение трех лет преследовал подростка в надежде совратить его. Как сообщили “МК” в следственном отделе по Хорошевскому району СУ СК при Прокуратуре РФ по Москве, 26 августа 2006 года 13-летний Павел вместе с другом отдыхал на берегу Москвы–реки рядом с Крылатским мостом. Там к ребятам подошел мужчина. Новый знакомый представился Евгением и предложил вместе пообедать в ресторане быстрого питания, а потом искупаться. Именно там, на берегу, педагог начал приставать к мальчику. Школьник убежал, но педофил успел снять голого подростка на мобильный телефон. После этого жизнь мальчика превратилась в сущий ад. Евгений узнал, где живет Павел, и стал шантажировать его. Этот кошмар продолжался три года. Преподаватель угрожал показать фотографию друзьям и знакомым Павла. Негодяй исписал непотребными надписями стены подъезда, где проживали друзья школьника. В один из дней он приехал в Сергиев Посад, к бабушке мальчика, и там накинулся на школьника с ножом. Наконец, отчаявши

Проверим какие значения принимает столбец 'topic':

In [ ]:
np.unique(dataset["train"]["topic"])

array(['auto', 'culture', 'daily', 'economics', 'editions', 'incident',
       'moscow', 'mosobl', 'nasha-moskva', 'new-year-2016', 'politics',
       'science', 'social', 'specprojects', 'sport', 'zloba-dnya'],
      dtype='<U13')

Т.к. перед нами стоит задача научить GPT по русскоязычным текстам новостей писать заголовки к ним, следует удалить лишние колонки: 'summary', 'topic', 'url', 'date'.

In [ ]:
# Столбцы, которые нужно удалить
columns_to_remove = ['summary', 'topic', 'url', 'date']

# Удаляем столбцы во всех частях датасета (train/val/test)
dataset = dataset.remove_columns(columns_to_remove)

# Проверяем оставшиеся столбцы
print(dataset["train"].column_names)

['text', 'title']


Проверим на null значения:

In [ ]:
for split in dataset:
    print(f"\nSplit: {split}")
    for column in dataset[split].column_names:
        null_count = sum(1 for item in dataset[split][column] if item is None)
        print(f"Столбец '{column}': {null_count} null значений")


Split: train
Столбец 'text': 0 null значений
Столбец 'title': 0 null значений

Split: validation
Столбец 'text': 0 null значений
Столбец 'title': 0 null значений

Split: test
Столбец 'text': 0 null значений
Столбец 'title': 0 null значений


Уменьшаем train до 1000 строк (выбираем первые 10000):

In [ ]:
small_train = dataset["train"].select(range(10000))

# Создаем новый DatasetDict с уменьшенным train
df = DatasetDict({
    "train": small_train,
    "validation": dataset["validation"],  # валидация без изменений
    "test": dataset["test"]              # тест без изменений
})

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'title'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['text', 'title'],
        num_rows: 750
    })
    test: Dataset({
        features: ['text', 'title'],
        num_rows: 757
    })
})

Отлично! Null значений нет.

## Подготовка обучающих данных

Подготовка данных

In [ ]:
def prepare_examples(examples):
    texts = examples["text"]
    titles = examples["title"]
    inputs = [f"{text}\n\nЗаголовок: {title}<|endoftext|>" for text, title in zip(texts, titles)]
    return {"formatted": inputs}

In [ ]:
tokenized_datasets = {
    "train": df["train"].map(prepare_examples, batched=True, remove_columns=["text", "title"]),
    "validation": df["validation"].map(prepare_examples, batched=True, remove_columns=["text", "title"])
}

Токенизация

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["formatted"], truncation=True, max_length=512)

In [ ]:
tokenized_datasets["train"] = tokenized_datasets["train"].map(tokenize_function, batched=True, remove_columns=["formatted"])
tokenized_datasets["validation"] = tokenized_datasets["validation"].map(tokenize_function, batched=True, remove_columns=["formatted"])

DataCollator для языкового моделирования

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Мы не используем masked language modeling
)

Оичщаем память:

In [ ]:
del dataset, small_train

## Обучение

In [ ]:
training_args = TrainingArguments(
    output_dir="./finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    fp16=True if DEVICE == "cuda" else False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

Запускаем обучение:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nesterenkoms2001 (nesterenkoms2001-digitaltech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.020600,3.031713
2,2.777200,3.038888
3,2.593900,3.060896


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=7500, training_loss=2.8386087890625, metrics={'train_runtime': 12907.5867, 'train_samples_per_second': 2.324, 'train_steps_per_second': 0.581, 'total_flos': 2.7811328093208576e+16, 'train_loss': 2.8386087890625, 'epoch': 3.0})

## Сохранение модели

In [ ]:
# Сохранение модели и токенизатора
model.save_pretrained("./news_title_generator")
tokenizer.save_pretrained("./news_title_generator")

## Загружаем модель

In [ ]:
DEVICE = torch.device("cuda:0")

# Путь к сохранённой модели
model_path = "./news_title_generator"

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)

# Проверка загрузки
print("Модель и токенизатор успешно загружены!")
print(f"Архитектура модели: {model.__class__.__name__}")

Модель и токенизатор успешно загружены!
Архитектура модели: GPT2LMHeadModel


## Тестирование

In [ ]:
def generate_title(text, max_new_tokens=50):
    # Формируем промпт с явным разделителем
    prompt = f"Текст: {text}\nЗаголовок:"
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True).to(DEVICE)

    # Генерация с явным указанием токенов
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Декодируем и очищаем вывод
    full_output = tokenizer.decode(output[0], skip_special_tokens=True)
    title = full_output.replace(prompt, "").strip()

    # Удаляем возможные HTML-теги и специальные символы
    title = title.split('<|endoftext|>')[0].split('</p>')[0].strip()

    return title

Выполним проверку на тестовых примерах из датасета:

In [ ]:
def generate_comparison_table(dataset_dict):
    # Преобразуем тестовый набор данных в DataFrame
    df = dataset_dict['test'].to_pandas()

    # Функция для обрезки текста до 100 слов
    def truncate_to_100_words(text):
        words = text.split()[:100]  # Берем первые 100 слов
        return ' '.join(words)

    # Применяем обрезку ко всем текстам
    df['text'] = df['text'].apply(truncate_to_100_words)

    # Выбираем 5 случайных примеров из датасета
    random_samples = df.sample(n=10)

    # Создаем строки для вывода
    output_lines = []

    for i, (_, row) in enumerate(random_samples.iterrows(), 1):
        original_text = row['text']
        original_title = row['title']
        predicted_title = generate_title(original_text)

        output_lines.append(f"Пример {i}")
        output_lines.append(f"Оригинальный текст: {original_text}")
        output_lines.append(f"Оригинальный заголовок: {original_title}")
        output_lines.append(f"Предсказанный заголовок: {predicted_title}")
        output_lines.append("")  # Пустая строка между примерами

    # Объединяем все строки с переносами
    return '\n'.join(output_lines)

In [ ]:
comparison_table = generate_comparison_table(df)
print(comparison_table)

Пример 1
Оригинальный текст: — Юлия Викторовна, расскажите для начала, в чем заключается основная задача логопеда? — Я бы сказала так: логопед — это главный специалист в дошкольном детстве. Логопед занимается не только развитием общей речевой активности, фонематического слуха, коррекцией звукопроизношения, накоплением словаря, развитием грамматической стороны речи, обучением навыкам словообразования, развитием связной речи, но и развитием психических процессов — внимание, память, восприятие, мышление, формирует предпосылки обучения грамоте, т.е. дает понятия «звук», «слово», «предложение», занимается развитием общей и мелкой моторики. Логопедия всегда стояла на стыке таких наук, как педагогика, психология, нейропсихология, психолингвистика, физиология и неврология. Для того чтобы скорректировать дефект, логопед должен обладать всеми этими
Оригинальный заголовок: Логопед рассказала, как воспитать умного ребенка
Предсказанный заголовок: Как научить ребенка говорить правильно?

Пример 2
О